# 4 - Feras Formidáveis

## 4.1 Quem classifica a classe classificadora?

### Introdução:

O objetivo desta atividade é alterar uma rede neural regressora feita em Python puro para resolver um problema de classificação. Treinou-se uma rede neural em um dataset simples sobre flores, utilizando a *binary cross entropy* como função de perda e demonstrando o funcionamento dessa rede. Boa parte do algoritmo foi adaptada do material de aula do Daniel Cassar [1]

### Desenvolvimento:

Importando as bibliotecas necessárias [2-5]:

In [1]:
import math
import random
import pandas as pd
import seaborn as sns


A nossa rede neural é baseada em uma estrutura de dados chamada **Valor** [1], em que definimos as operações matemáticas necessárias para o treinamento do modelo e o cálculo do gradiente. Adicionou-se a operação *log*, necessária o cálculo da função de perda:

In [48]:
class Valor:
    def __init__(self, data, progenitor=(), operador_mae="", rotulo=""):
        self.data = data
        self.progenitor = progenitor
        self.operador_mae = operador_mae
        self.rotulo = rotulo
        self.grad = 0

    def __repr__(self):
        return f"Valor(data={self.data})"
    
    def __add__(self, outro_valor):
        """Realiza a operação: self + outro_valor."""
        
        if not isinstance(outro_valor, Valor):
            outro_valor = Valor(outro_valor)
            
        progenitor = (self, outro_valor)
        data = self.data + outro_valor.data
        operador_mae = "+"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_adicao():
            self.grad += resultado.grad
            outro_valor.grad += resultado.grad
            
        resultado.propagar = propagar_adicao
        
        return resultado
    
    def __mul__(self, outro_valor):
        """Realiza a operação: self * outro_valor."""
        
        if not isinstance(outro_valor, Valor):
            outro_valor = Valor(outro_valor)
            
        progenitor = (self, outro_valor)
        data = self.data * outro_valor.data
        operador_mae = "*"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_multiplicacao():
            self.grad += resultado.grad * outro_valor.data # grad_filho * derivada filho em relação a mãe
            outro_valor.grad += resultado.grad * self.data
            
        resultado.propagar = propagar_multiplicacao
        
        return resultado
    
    def exp(self):
        """Realiza a operação: exp(self)"""
        progenitor = (self, )
        data = math.exp(self.data)
        operador_mae = "exp"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_exp():
            self.grad += resultado.grad * data 
        
        resultado.propagar = propagar_exp
        
        return resultado

    def log(self):
        """Realiza a operação: ln(self)"""
        progenitor = (self, )
        data = math.log(self.data)
        operador_mae = "log"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_log():
            self.grad += resultado.grad / (self.data )
        
        resultado.propagar = propagar_log
        
        return resultado

    def __pow__(self, expoente):
        """Realiza a operação: self ** expoente"""
        assert isinstance(expoente, (int, float))
        progenitor = (self, )
        data = self.data ** expoente
        operador_mae = f"**{expoente}"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_pow():
            self.grad += resultado.grad * (expoente * self.data ** (expoente - 1))
        
        resultado.propagar = propagar_pow
        
        return resultado
    
    def __truediv__(self, outro_valor):
        """Realiza a operação: self / outro_valor"""
        return self * outro_valor ** (-1)
    
    def __neg__(self):
        """Realiza a operação: -self"""
        return self * -1
    
    def __sub__(self, outro_valor):
        """Realiza a operação: self - outro_valor"""
        return self + (-outro_valor)
    
    def __radd__(self, outro_valor):
        """Realiza a operação: outro_valor + self"""
        return self + outro_valor
    
    def __rmul__(self, outro_valor):
        """Realiza a operação: outro_valor * self"""
        return self * outro_valor
    
    def sig(self):
        """Realiza a operação: exp(self) / (exp(self) + 1)"""
        return self.exp() / (self.exp() + 1)
    
    def propagar(self):
        pass
    
    def propagar_tudo(self):
        
        self.grad = 1
        
        ordem_topologica = []
        
        visitados = set()

        def constroi_ordem_topologica(v):
            if v not in visitados:
                visitados.add(v)
                for progenitor in v.progenitor:
                    constroi_ordem_topologica(progenitor)
                ordem_topologica.append(v)

        constroi_ordem_topologica(self)
        
        for vertice in reversed(ordem_topologica):
            vertice.propagar()

Nossa rede é composta por neurônios, em que recebe informações de entrada, aplica uma função de ativação e passa a informação adiante, sendo definida na classe Neuronio [1]

In [4]:
class Neuronio:
    def __init__(self, num_dados_entrada):
        self.vies = Valor(random.uniform(-1, 1))
        
        self.pesos = []
        for i in range(num_dados_entrada):
            self.pesos.append(Valor(random.uniform(-1, 1)))
            
    def __call__(self, x):
        
        assert len(x) == len(self.pesos)
        
        soma = 0
        for info_entrada, peso_interno in zip(x, self.pesos):
            soma += info_entrada * peso_interno
            
        soma += self.vies  
        dado_de_saida = soma.sig()
        
        return dado_de_saida       
    
    def parametros(self):
        return self.pesos + [self.vies]

Definindo a classe Camada [1], em que os neurônios são agrupados e passando a informação através de camadas ocultas:

In [5]:
class Camada:
    def __init__(self, num_neuronios, num_dados_entrada):
        neuronios = []
        
        for _ in range(num_neuronios):
            neuronio = Neuronio(num_dados_entrada)
            neuronios.append(neuronio)
            
        self.neuronios = neuronios     
        
    def __call__(self, x):
        dados_de_saida = []
        
        for neuronio in self.neuronios:
            informacao = neuronio(x)
            dados_de_saida.append(informacao)
            
        if len(dados_de_saida) == 1:
            return dados_de_saida[0]
        else:        
            return dados_de_saida  
    
    def parametros(self):
        params = []
        
        for neuronio in self.neuronios:
            params_neuronio = neuronio.parametros()
            params.extend(params_neuronio)
        
        return params

Por fim, definindo nossa rede neural classificadora, que vai organizar nossos dados em camadas 

In [6]:
class Classifier_nn:
    def __init__(self, num_dados_entrada, num_neuronios_por_camada):
        
        percurso = [num_dados_entrada] + num_neuronios_por_camada
        
        camadas = []
        
        for i in range(len(num_neuronios_por_camada)):
            camada = Camada(num_neuronios_por_camada[i], percurso[i])
            camadas.append(camada)
            
        self.camadas = camadas
        
    def __call__(self, x):
        for camada in self.camadas:
            x = camada(x)
        return x
    
    def parametros(self):
        params = []
        
        for camada in self.camadas:
            parametros_camada = camada.parametros()
            params.extend(parametros_camada)
            
        return params

Carregando o dataset próprio da biblioteca *Seaborn* relacionado a plantas [6], contendo características como comprimento e largura das pétalas e sépalas de uma flor, indicando a espécie da planta

In [7]:
df = sns.load_dataset('iris')
logica = df["species"] != 'versicolor'
df_binario = df.loc[logica]
df_binario

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


Utilizou-se apenas 2 das 3 espécies fornecidas no dataset (*setosa* e *virginica*) para esse problema de classificação, convertendo as espécies para número, em que setosa representa 0 e virginica 1

In [8]:
for especie in df_binario['species']:
    if especie == 'setosa':
        df_binario['species'] = df_binario['species'].replace(especie, 0)
    elif especie == 'virginica':
        df_binario['species'] = df_binario['species'].replace(especie, 1)

df_binario

C:\Users\enzo24035\AppData\Local\Temp\ipykernel_33324\3480997974.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_binario['species'] = df_binario['species'].replace(especie, 0)
C:\Users\enzo24035\AppData\Local\Temp\ipykernel_33324\3480997974.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_binario['species'] = df_binario['species'].replace(especie, 1)


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,1
146,6.3,2.5,5.0,1.9,1
147,6.5,3.0,5.2,2.0,1
148,6.2,3.4,5.4,2.3,1


Separando o dataset em dados de entrada e os valores reais, a fim de comparar com as previsões do modelo, além de definir uma rede neural classificadora com base na arquitetura da rede escolhida. De acordo com o dataset, há 4 atributos do problema, sendo os dados de entrada, e o algoritmo tem 1 dado de saída contendo a previsão da espécie (valor entre 0 e 1, sendo setosa mais próximo de 0 e virginica mais próximo de 1). Note que o valor real *y_true* foi atribuido na classe Valor, para permitir calcular a função de perda e os gradientes locais.

In [49]:
x = df_binario.drop("species", axis=1).values
y_true = df_binario['species'].values
y_true = [Valor(value) for value in y_true]

NUM_DADOS_DE_ENTRADA = 4  
NUM_DADOS_DE_SAIDA = 1    
CAMADAS_OCULTAS = [3, 2]  

arquitetura_da_rede = CAMADAS_OCULTAS + [NUM_DADOS_DE_SAIDA]

meu_classificador = Classifier_nn(NUM_DADOS_DE_ENTRADA, arquitetura_da_rede)

Fazendo a etapa de treinamento para um número de épocas como 50 e uma taxa de aprendizado de 0.001. A função de perda usada é a *binary cross entropy*, própria para problemas de classificação [7], sendo que a cada época a perda é mostrada

In [55]:
NUM_EPOCAS = 50
TAXA_DE_APRENDIZADO = 0.001

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = []
    for exemplo in x:
        previsao = meu_classificador(exemplo)
        y_pred.append(previsao)

    # loss
    erros = []
    for yt, yp in zip(y_true, y_pred):
        variavel = Valor(1 - yp.data)
        erro = -(yt.data * yp.log() + (1 - yt.data) * variavel.log())
        erros.append(erro)

    loss = sum(erros) / len(y_pred)
    
    # zero grad
    for p in meu_classificador.parametros():
        p.grad = 0

    # backpropagation
    loss.propagar_tudo()

    # atualiza parâmetros
    for p in meu_classificador.parametros():
        p.data = p.data - p.grad * TAXA_DE_APRENDIZADO

    # mostra resultado (opcional)
    print(epoca, loss.data)

0 0.7979918960917908
1 0.7980459264345432
2 0.7980999611222487
3 0.7981540001498348
4 0.7982080435122337
5 0.7982620912043776
6 0.7983161432212039
7 0.7983701995576512
8 0.7984242602086623
9 0.7984783251691847
10 0.7985323944341656
11 0.7985864679985565
12 0.7986405458573141
13 0.7986946280053949
14 0.7987487144377617
15 0.7988028051493768
16 0.798856900135207
17 0.7989109993902253
18 0.7989651029094021
19 0.7990192106877158
20 0.7990733227201443
21 0.7991274390016706
22 0.7991815595272803
23 0.7992356842919607
24 0.7992898132907054
25 0.7993439465185072
26 0.7993980839703639
27 0.7994522256412764
28 0.799506371526249
29 0.7995605216202876
30 0.7996146759184012
31 0.7996688344156038
32 0.7997229971069103
33 0.7997771639873388
34 0.7998313350519127
35 0.7998855102956569
36 0.7999396897135972
37 0.7999938733007659
38 0.8000480610521974
39 0.8001022529629264
40 0.8001564490279951
41 0.8002106492424439
42 0.8002648536013212
43 0.8003190620996736
44 0.8003732747325544
45 0.8004274914950184


Percebe-se que embora a perda seja relativamente pequena, ela cresce ligeiramente ao longo das épocas, indicando que o algoritmo não está tão bem otimizado para o problema de classificação

Mostrando os valores preditos pela rede neural:

In [54]:
y_pred

[Valor(data=0.7186423436735064),
 Valor(data=0.7188961397162188),
 Valor(data=0.718462382863991),
 Valor(data=0.7190360044535538),
 Valor(data=0.7185704389693751),
 Valor(data=0.7189365906048417),
 Valor(data=0.7184024060500165),
 Valor(data=0.7189108262543527),
 Valor(data=0.718924295062161),
 Valor(data=0.7192232727144431),
 Valor(data=0.718861825998755),
 Valor(data=0.7191022843915243),
 Valor(data=0.7190541163445865),
 Valor(data=0.7181466896506126),
 Valor(data=0.7181589969250736),
 Valor(data=0.7184604986086357),
 Valor(data=0.7179511940692376),
 Valor(data=0.7184718269059072),
 Valor(data=0.7191631700619531),
 Valor(data=0.7185938970447524),
 Valor(data=0.7193795973871049),
 Valor(data=0.7184695823102513),
 Valor(data=0.7171983318474485),
 Valor(data=0.7189732547046654),
 Valor(data=0.7196684856061768),
 Valor(data=0.7193560654398115),
 Valor(data=0.7188289140118644),
 Valor(data=0.7189004413248585),
 Valor(data=0.7187138663754578),
 Valor(data=0.7192067198231527),
 Valor(data=0

Note que todos os valores estão bem próximos e em apenas uma categoria, indicando que o modelo só previu dados para o rótulo 0 (espécie setosa), mesmo com os dados de entrada contendo plantas das 2 espécies

### Conclusão:

Foi possível treinar uma rede neural em python puro para tarefas de classificação, embora ela não esteja bem otimizada, aumentando a perda ao longo do treinamento e prevendo apenas um rótulo para os dados do problema. Mesmo assim, foi interessante aprender sobre esse tipo de problema, aumentando meu conhecimento.

### Referências:

[1] CASSAR, Daniel. "ATP-303 NN 4.2 - Notebook MLP.ipynb". Material de Aula, 2025.

[2] Biblioteca Math. https://docs.python.org/3/library/math.html

[3] Biblioteca Random. https://docs.python.org/3/library/random.html

[4] Biblioteca Pandas. https://pandas.pydata.org/docs/user_guide/index.html#user-guide

[5] Biblioteca Seaborn. https://seaborn.pydata.org/

[6] Dataset estudado. https://archive.ics.uci.edu/dataset/53/iris

[7] Função de perda analisada. https://medium.com/ensina-ai/uma-explicação-visual-para-função-de-custo-binary-cross-entropy-ou-log-loss-eaee662c396c